In [ ]:
#!pip install smartsheet-python-sdk

In [ ]:
#!pip install pubchempy

In [ ]:
#!pip install jupyterthemes

### Load in Chem Dict

In [19]:
import smartsheet
import pandas as pd
import numpy as np
# Initialize client
smartsheet_client = smartsheet.Smartsheet('3gxh6y5CzsT3lEkvkUny0YVdplz6qba8pv8WH')

# Specify the sheet ID
sheet_id = '8604420338044804'

# Load entire sheet
sheet = smartsheet_client.Sheets.get_sheet(sheet_id)

# Convert sheet to DataFrame
columns = [col.title for col in sheet.columns]
rows = []
for row in sheet.rows:
    row_data = []
    for cell in row.cells:
        row_data.append(cell.value)
    rows.append(row_data)

chemdict = pd.DataFrame(rows, columns=columns)
chemdict = chemdict.rename(columns={'substances': 'substance'})

# Save DataFrame to CSV
#chemdict = chemdict[['substances', 'PubChemCID', 'sixwords', 'notes', 'smiles1', 'AI_tags','tags']]

chemdict.head()

,substance_ID,assignment,substance,PubChemCID,CAS,synonyms,chemical,pharmco,street_smarts,trending,...,R1 complete,tags,receptor,opioid,stimulant,psychedelic,cannabinoid,sedative,steroid,category
0,1.0,Nab,"1,2-Dibromo-4,5-methylenedioxybenzene",225814.0,5279-32-3,"5,6-dibromo-1,3-benzodioxole;1,2-dibromo-4,5-(methylenedioxy)benzene","impurity, methamphetamine impurity, stimulant, synthetic",human effects uncertain,None,uncommon,...,to do,impurity;methamphetamine impurity;stimulant;synthetic;human effects uncertain;uncommon;not psychoactive;,None,None,stimulant,None,None,None,None,stimulant
1,2.0,Nab,"1,3-Diacetin",66924.0,105-70-4,"glyceryl diacetate;2-Hydroxypropane-1,3-diyl diacetate;glyceryl 1,3-diacetate",fentanyl impurity,"human effects uncertain, inert","cut, flavor, non-toxic","established, uncommon",...,done,fentanyl impurity;human effects uncertain;inert;cut;flavor;non-toxic;established;uncommon;local relevance;,None,None,None,None,None,None,None,other
2,3.0,Nab,"1,4-Butanediol",8064.0,110-63-4,None,"industrial chemical, solvent, synthetic",None,downer,None,...,done,industrial chemical;solvent;synthetic;downer;GHB impurity;night life;,None,None,None,None,None,None,None,other
3,4.0,Anuja,"1-(3,4-Methylenedioxyphenyl)-2-propanol",95505.0,None,None,MDMA impurity,None,None,None,...,None,MDMA impurity;,None,None,None,None,None,None,None,None
4,5.0,Nab,1-2-propanol,7900.0,None,"3,4-Methylenedioxyphenyl",None,human effects uncertain,None,None,...,to do,human effects uncertain;,None,None,None,None,None,None,None,None


#### 1. select substances that have been tagged by experts

In [20]:
chemdict_cleaned = chemdict[chemdict['assignment'] != 'Anuja']
chemdict_cleaned = chemdict_cleaned.dropna(subset=['tags'])

#only select these columns
chemdict_cleaned = chemdict_cleaned[['substance', 'PubChemCID', 'sixwords', 'notes', 'smiles1', 'AI_tags','tags', 'category']]
chemdict_cleaned.head()

,substance,PubChemCID,sixwords,notes,smiles1,AI_tags,tags,category
0,"1,2-Dibromo-4,5-methylenedioxybenzene",225814.0,"Meth production impurity, rare",None,C1OC2=CC(=C(C=C2O1)Br)Br,None,impurity;methamphetamine impurity;stimulant;synthetic;human effects uncertain;uncommon;not psychoactive;,stimulant
1,"1,3-Diacetin",66924.0,Inactive fentanyl cut common in NC,"Common in fentanyl samples in NC, rare elsewhere. Food, paint, other additive as solvent, flavor, plasticizer.",CC(=O)OCC(COC(=O)C)O,"impurity, flavor, GCMS breakdown product",fentanyl impurity;human effects uncertain;inert;cut;flavor;non-toxic;established;uncommon;local relevance;,other
2,"1,4-Butanediol",8064.0,"related to GHB, unclear if active","Precurser to gamma-hydroxybutyrate (GHB) - metabolized in the body, so hypothesized active, but mixed reports. Some federal courts in the USA have stated that 1,4-butanediol exerts effects similar to its metabolite GHB, but several other federal courts have ruled that it does not.",C(CCO)CO,"impurity, sedative, downer",industrial chemical;solvent;synthetic;downer;GHB impurity;night life;,other
4,1-2-propanol,7900.0,None,None,CC(COC)O,None,human effects uncertain;,None
6,1-[methyl]cyclopentanol,73830.0,"Leftover from making ketamine, inactive",Appears similar to other starting materials used in clandestine manufacture of ketamine https://www.sciencedirect.com/science/article/abs/pii/S0379073823002268,CC1(CCCC1)O,"impurity, GCMS breakdown product, uncommon",impurity;ketamine impurity;synthetic;human effects uncertain;uncommon;not psychoactive;synthesis byproduct;,other


#### 2. combine Ai tags with tags into 1 column

In [21]:
# I want to combine the AI_tags with the tags column and have no repetitions

#firstly, replace the , with ; in AI_tags
chemdict_cleaned['AI_tags'] = chemdict_cleaned['AI_tags'].str.replace(',', ';')

#now i want to combine the two columns together
chemdict_cleaned['tags_combined'] = chemdict_cleaned['AI_tags'].fillna('') + ';' + chemdict_cleaned['tags']

#remove empty spaces before
chemdict_cleaned['tags_combined'] = chemdict_cleaned['tags_combined'].str.replace(r'\s*;\s*', ';', regex=True)

# remove the repetitions within the column
def remove_duplicates_from_column(df, column_name):
    df[column_name] = df[column_name].astype(str).apply(lambda x: ';'.join(sorted(list(set(x.split(';'))))))
    return df

chemdict_cleaned = remove_duplicates_from_column(chemdict_cleaned, 'tags_combined')

#drop the original tags columns
chemdict_cleaned = chemdict_cleaned[['substance', 'PubChemCID', 'sixwords', 'notes', 'smiles1', 'category', 'tags_combined']]
chemdict_cleaned.head()

,substance,PubChemCID,sixwords,notes,smiles1,category,tags_combined
0,"1,2-Dibromo-4,5-methylenedioxybenzene",225814.0,"Meth production impurity, rare",None,C1OC2=CC(=C(C=C2O1)Br)Br,stimulant,;human effects uncertain;impurity;methamphetamine impurity;not psychoactive;stimulant;synthetic;uncommon
1,"1,3-Diacetin",66924.0,Inactive fentanyl cut common in NC,"Common in fentanyl samples in NC, rare elsewhere. Food, paint, other additive as solvent, flavor, plasticizer.",CC(=O)OCC(COC(=O)C)O,other,;GCMS breakdown product;cut;established;fentanyl impurity;flavor;human effects uncertain;impurity;inert;local relevance;non-toxic;uncommon
2,"1,4-Butanediol",8064.0,"related to GHB, unclear if active","Precurser to gamma-hydroxybutyrate (GHB) - metabolized in the body, so hypothesized active, but mixed reports. Some federal courts in the USA have stated that 1,4-butanediol exerts effects similar to its metabolite GHB, but several other federal courts have ruled that it does not.",C(CCO)CO,other,;GHB impurity;downer;impurity;industrial chemical;night life;sedative;solvent;synthetic
4,1-2-propanol,7900.0,None,None,CC(COC)O,None,;human effects uncertain
6,1-[methyl]cyclopentanol,73830.0,"Leftover from making ketamine, inactive",Appears similar to other starting materials used in clandestine manufacture of ketamine https://www.sciencedirect.com/science/article/abs/pii/S0379073823002268,CC1(CCCC1)O,other,;GCMS breakdown product;human effects uncertain;impurity;ketamine impurity;not psychoactive;synthesis byproduct;synthetic;uncommon


### 3. find out a way to sift through six words and notes columns to extract valuable tags

In [22]:
#i have the sixwords column that I also want to put into a 'tag' format
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

chemdict_cleaned[['sixwords', 'notes', 'tags_combined']].head()


,sixwords,notes,tags_combined
0,"Meth production impurity, rare",None,;human effects uncertain;impurity;methamphetamine impurity;not psychoactive;stimulant;synthetic;uncommon
1,Inactive fentanyl cut common in NC,"Common in fentanyl samples in NC, rare elsewhere. Food, paint, other additive as solvent, flavor, plasticizer.",;GCMS breakdown product;cut;established;fentanyl impurity;flavor;human effects uncertain;impurity;inert;local relevance;non-toxic;uncommon
2,"related to GHB, unclear if active","Precurser to gamma-hydroxybutyrate (GHB) - metabolized in the body, so hypothesized active, but mixed reports. Some federal courts in the USA have stated that 1,4-butanediol exerts effects similar to its metabolite GHB, but several other federal courts have ruled that it does not.",;GHB impurity;downer;impurity;industrial chemical;night life;sedative;solvent;synthetic
4,None,None,;human effects uncertain
6,"Leftover from making ketamine, inactive",Appears similar to other starting materials used in clandestine manufacture of ketamine https://www.sciencedirect.com/science/article/abs/pii/S0379073823002268,;GCMS breakdown product;human effects uncertain;impurity;ketamine impurity;not psychoactive;synthesis byproduct;synthetic;uncommon


#### 4. make tags into dummy variables

In [23]:
def create_dummy_variables(df, column_name):

    # Create a copy of the input DataFrame
    result_df = df.copy()
    
    # Split the strings and get unique values
    unique_values = set()
    for items in df[column_name].str.split(';').dropna():
        unique_values.update(items)
    unique_values = sorted(list(unique_values))
    
    # Create a numpy array to store all dummy values at once
    n_samples = len(df)
    n_features = len(unique_values)
    dummy_matrix = np.zeros((n_samples, n_features))
    dummy_matrix = dummy_matrix.astype(int)

    
    # Fill the dummy matrix
    for idx, row in enumerate(df[column_name]):
        if pd.notna(row):
            categories = str(row).split(';')
            for cat in categories:
                if cat in unique_values:
                    col_idx = unique_values.index(cat)
                    dummy_matrix[idx, col_idx] = 1
    
    # Create a DataFrame from the dummy matrix
    dummy_df = pd.DataFrame(dummy_matrix, columns=unique_values, index=df.index)
    
    # Combine the original DataFrame (excluding the processed column) with dummy columns
    result_df = pd.concat([df.drop(column_name, axis=1), dummy_df], axis=1)
    
    return result_df

In [27]:
dummy_frame = create_dummy_variables(chemdict_cleaned, 'tags_combined')
dummy_frame.head()

KeyError: "None of [Index(['ketamine'], dtype='object')] are in the [columns]"

In [25]:
tags = list(dummy_frame.columns.values)
tags.sort()
print(tags)

['', '2CB impurity', 'DMT impurity', 'GCMS breakdown product', 'GHB impurity', 'LSD impurity', 'MDMA impurity', 'NMDA receptor antagonist', 'NSAID', 'Opioid use disorder treatment', 'PCP-related', 'PubChemCID', 'ace inhibitor', 'active agent', 'addictive', 'adulterant', 'alcohol use treatment', 'alkaloid', 'alpha-2 agonist', 'amphetamine & derivatives', 'anabolic', 'analgesic', 'anesthetic', 'anti-anxiety', 'anti-cholinergic', 'anti-convulsant', 'anti-depressant', 'anti-depressant impurity', 'anti-malarial', 'anti-microbial', 'anti-parasitic', 'anti-psychotic', 'anti-tussive', 'antihistamine', 'appetite suppressant', 'artificial sweetener', 'arylcyclohexylamine', 'atypical benzodiazepine', 'barbiturate', 'benzodiazepine', 'benzodiazepine metabolite', 'birth control', 'bittering agent', 'bulking agent', 'cannabiniod', 'cannabinoid', 'category', 'cocaine cutting agent', 'cocaine impurity', 'codeine derivative', 'common', 'concern', 'contaminant', 'cut', 'cutting agent', 'cyclohexamine', 

#### clean up value names

In [11]:
dummy_frame['opioid use treatment'] = dummy_frame['Opioid use disorder treatment'] + dummy_frame['opioid use disorder treatment'] + dummy_frame['opioid use treatment']
dummy_frame['cannabinoid'] = dummy_frame['cannabiniod'] + dummy_frame['cannabinoid']
dummy_frame['entactogen'] = dummy_frame['enactogen'] + dummy_frame['entactogen']
dummy_frame['prescription'] = dummy_frame['perscription'] + dummy_frame['prescription uses'] + dummy_frame['prescription']
dummy_frame['tryptamine'] = dummy_frame['tryptamine'] + dummy_frame['typtamine']
dummy_frame['cut'] = dummy_frame['cut'] + dummy_frame['cutting agent']

In [16]:
remove_cols = ['Opioid use disorder treatment', 'opioid use disorder treatment',
               'cannabiniod',
               'enactogen', 
                'perscription', 'prescription uses',
               'tryptamine',
               'cutting agent']
dummy_frame = dummy_frame.drop(columns=remove_cols)
dummy_frame.head()

,substance,PubChemCID,sixwords,notes,smiles1,category,,2CB impurity,DMT impurity,GCMS breakdown product,...,tramadol impurity,typtamine,uncertain,uncommon,up-and-down,upper,vasodilator,veterinary,vitamin,weak opioid
0,"1,2-Dibromo-4,5-methylenedioxybenzene",225814.0,"Meth production impurity, rare",None,C1OC2=CC(=C(C=C2O1)Br)Br,stimulant,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,"1,3-Diacetin",66924.0,Inactive fentanyl cut common in NC,"Common in fentanyl samples in NC, rare elsewhere. Food, paint, other additive as solvent, flavor, plasticizer.",CC(=O)OCC(COC(=O)C)O,other,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,"1,4-Butanediol",8064.0,"related to GHB, unclear if active","Precurser to gamma-hydroxybutyrate (GHB) - metabolized in the body, so hypothesized active, but mixed reports. Some federal courts in the USA have stated that 1,4-butanediol exerts effects similar to its metabolite GHB, but several other federal courts have ruled that it does not.",C(CCO)CO,other,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1-2-propanol,7900.0,None,None,CC(COC)O,None,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1-[methyl]cyclopentanol,73830.0,"Leftover from making ketamine, inactive",Appears similar to other starting materials used in clandestine manufacture of ketamine https://www.sciencedirect.com/science/article/abs/pii/S0379073823002268,CC1(CCCC1)O,other,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0


#### 5. add in cut vs adulterant vs impurity information (into 1 columns)

In [17]:
dummy_frame.to_csv('dummy_tags.csv', index=False)  # index=False prevents writing the index

In [28]:
dummy_frame[['ketamine']]

KeyError: "None of [Index(['ketamine'], dtype='object')] are in the [columns]"